# Ejercicio QKD
Universidad del Valle de Guatemala<br>
Cifrado de Información<br>
Pablo Andrés Zamora Vásquez<br>
Carné 21780



In [1]:
import pandas as pd
import random

In [14]:
# Simulación de 15 rondas del protocolo BB84
num_rounds = 15

# Función para lanzar moneda (0 o 1 aleatorio)
def coin_flip():
    return random.choice([0, 1])

# Inicialización de listas
rows = []

for i in range(1, num_rounds + 1):
    bit_alice = coin_flip()
    base_alice = coin_flip()  # 0: recta (↕), 1: diagonal (↗)
    base_alice_sim = "↕" if base_alice == 0 else "↗"

    # Según la base y el bit, se define la polarización del fotón enviado
    # ↕: 0 = →, 1 = ↑
    # ↗: 0 = ↗, 1 = ↖
    
    if base_alice == 0:  # Base recta
        sent_photon = "→" if bit_alice == 0 else "↑"
    else:  # Base diagonal
        sent_photon = "↗" if bit_alice == 0 else "↖"

    base_bob = coin_flip()
    base_bob_sim = "↕" if base_bob == 0 else "↗"

    # Bit recibido depende de si base coincide o no
    if base_alice == base_bob: # Si las bases coinciden, el bit recibido es el mismo que el enviado el 100% de las veces
        received_bit = bit_alice
        bases_match = "sí"
        use_bit = "sí"
    else: # Si las bases no coinciden, hay un 50% de probabilidad de que el bit recibido sea el correcto
        received_bit = coin_flip()
        bases_match = "no"
        use_bit = "no"

    rows.append({
        "N°": i,
        "Bit de Alice": bit_alice,
        "Base de Alice (↕/↗)": base_alice_sim,
        "Fotón enviado": sent_photon,
        "Base de Bob (↕/↗)": base_bob_sim,
        "Bit recibido": received_bit,
        "¿Bases coinciden?": bases_match,
        "¿Usar bit?": use_bit
    })

# Crear DataFrame
df_bb84 = pd.DataFrame(rows)
df_bb84

,N°,Bit de Alice,Base de Alice (↕/↗),Fotón enviado,Base de Bob (↕/↗),Bit recibido,¿Bases coinciden?,¿Usar bit?
0,1,1,↕,↑,↗,0,no,no
1,2,1,↗,↖,↕,1,no,no
2,3,0,↕,→,↕,0,sí,sí
3,4,1,↕,↑,↕,1,sí,sí
4,5,1,↕,↑,↕,1,sí,sí
5,6,0,↕,→,↗,1,no,no
6,7,1,↗,↖,↕,1,no,no
7,8,0,↕,→,↗,1,no,no
8,9,1,↗,↖,↕,0,no,no
9,10,1,↗,↖,↗,1,sí,sí


## Preguntas de Análisis

1. ¿Cuántos bits finales obtuvieron de la clave?

In [15]:
# Cuántos bits se obtuvieron para la clave
num_bits_key = df_bb84[df_bb84["¿Usar bit?"] == "sí"].shape[0]
print(f"\nNúmero de bits obtenidos para la clave: {num_bits_key}")


Número de bits obtenidos para la clave: 8


2. ¿Qué porcentaje representa respecto al total?

In [16]:
porcentage_key = (num_bits_key / num_rounds) * 100
print(f"Porcentaje de bits obtenidos para la clave: {porcentage_key:.2f}%")

Porcentaje de bits obtenidos para la clave: 53.33%


3. ¿Qué pasaría si Eve interceptara los fotones y usara bases incorrectas?

Dado que Eve no conoce las bases que utiliza Alice para enviar cada bit, si utiliza la base incorrecta, cambia el estado original del fotón que debería recibir Bob. Por tanto, cuando Bob reciba este fotón, incluso si utiliza la misma base que Alice, medirá un bit incorrecto. Esto resulta en errores entre los bits compartidos entre Alice y Bob. Si estos se enteran de las discrepancias entre los bits, detectarán la presencia de Eve.

4. ¿Cómo se puede detectar su presencia?

Para detectar la presencia de Eve, Alice y Bob pueden realizar un chequeo de integridad. Es decir, seleccionan una fracción aleatoria de los bits donde sus bases coincidieron y comparan públicamente sus valores; si hay diferencias, es porque hay alguien capturando los fotones.

5. ¿Qué ventajas y desventajas tiene este protocolo frente a otros cifrados tradicionales?

Entre las ventajas que presenta este protocolo frente a la criptografía tracional se encuentran:
- Su seguridad se basa en leyes físicas en lugar de la complejidad matemática. En particular, aprovecha principios como la superposición y el colapso del estado cuántico, lo que asegura que cuando las bases del emisor y receptor coinciden, el receptor obtiene el bit correcto; si no coinciden, la probabilidad de acertar es del 50%, introduciendo la aleatoriedad necesaria para proteger la información.

- Permite detectar la presencia de un espía gracias a que, según la mecánica cuántica, medir un fotón altera su estado. Esto significa que cualquier intento de interceptación produce errores detectables en una verificación posterior.

- Además, el protocolo se beneficia del principio de no clonación: un fotón no puede copiarse sin cambiar su estado, lo que hace imposible duplicarlo sin dejar rastro.

Por otro lado, entre las desventajas que presenta este protocolo están:
- Requiere tecnología cuántica especializada, que actualmente es costosa y compleja de implementar a gran escala.

- Es necesario establecer una línea directa (por ejemplo, fibra óptica dedicada o enlace óptico) y sincronización precisa entre emisor y receptor.

- Es sensible a fuentes de ruido como la pérdida de fotones, la ineficiencia de los detectores y las fluctuaciones en la fase del canal óptico, lo que puede limitar la distancia o velocidad de transmisión.

## Referencias

Balamurugan, N. Y., & Lingaraj, S. G. (2018). Quantum key distribution protocols and security analysis. *International Journal of Scientific Research in Computer Science, Engineering and Information Technology*, 3(6), 947–953. https://ijsrcseit.com/paper/CSEIT1836161.pdf

Quantum Zeitgeist. (n.d.). *The BB84 Protocol*. https://quantumzeitgeist.com/the-bb84-protocol/

MinutePhysics. (2016). *Quantum Cryptography: BB84 Protocol* [Video]. YouTube. https://www.youtube.com/watch?v=V3WzH2up7Os
